# Fitz-Hugh-Nagumo model

In [1]:
%matplotlib inline

import numpy as np
import scipy as sp
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import glob, os, psutil, time

p,n =  10000, 20
k,l = 1,1
T = 2000

# fix emission model parameters
pars = {}
pars['C'] = np.random.normal(size=(p,n))/np.sqrt(n)
pars['R'] = 1 * np.ones(p)
tf = 1


# diff. eq.
I = 0
def ddt_const(x):
    v,w = x[0],x[1]
    return np.array((v - v**3/3 -w + I, 0.08*(v +0.7-0.8*w)))

# quiver plot

N = 30
V, W = np.meshgrid(np.linspace(-3,3,N), np.linspace(-3,3,N))
dV, dW = np.zeros((N,N)), np.zeros((N,N))
for i in range(N):
    for j in range(N):
        tmp = ddt_const(np.array((V[i,j], W[i,j])))
        dV[i,j], dW[i,j] = tmp[0], tmp[1]

#plt.figure(figsize=(10,10))
#plt.quiver(V, W, dV, dW, 
#           edgecolor='k', linewidth=.5)
#plt.show()

# generate latent trajectory

I = [0]
def ddt(x,t):
    v,w = x[:n//2],x[n//2:]
    return np.hstack((v - v**3/3 -w + I[t], 0.08*(v +0.7-0.8*w)))
I0 = 0.3
ts = np.arange(0,tf*2000,tf*2000/(2*tf*T))
I = 0.01/0.2 * np.random.normal(size=(len(ts),n//2))*np.sqrt(0.1) + I0
x0 = np.random.normal(size=n)
x = sp.integrate.odeint(ddt, x0, ts)
#plt.figure(figsize=(20,10))
#plt.plot(ts, x[:,0:-1:2], 'k')
#plt.hold(True)
#plt.plot(ts, x[:,1:-1:2], 'b--')
#plt.show()

x = x[T:-1:tf] # downsample to 1s to reduce 1-time-lag correlation 
x = x[-T:]
x = (x - np.mean(x,0))/np.sqrt(np.var(x,0))
#T = T//tf      # (otherwise we'll need a lot of time-lags to add information)



/home/mackelab/anaconda3/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:
/home/mackelab/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:46: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [7]:
x0.shape, ddt(x0, ts[0]).shape, ts.shape

/home/mackelab/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:46: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


((20,), (20,), (4000,))

In [ ]:
# generate noisy observed data

# I/O matter
mmap, chunksize = False, np.min((T,100))
#data_path, save_file = '/media/marcel/636f7b46-1fd1-4600-b69e-86d2ed82002c/stitching/hankel/', 'test'
data_path, save_file = '../fits/', 'test'
pa, pb = np.min((p, 1000)), np.min((p, 1000))
idx_a, idx_b = np.sort(np.random.choice(p,pa,replace=False)), np.sort(np.random.choice(p,pb,replace=False))
verbose=True

T = x.shape[0]
if mmap:
    y = np.memmap(data_path+'y', dtype=np.float, mode='w+', shape=(T,p))
    eps = np.memmap(data_path+'eps', dtype=np.float, mode='w+', shape=(T,p))
else:
    y = np.empty(shape=(T,p))
    eps = np.empty(shape=(T,p))
    
for i in range(T//chunksize):
    idx = range(chunksize*i,chunksize*(i+1))
    eps[idx] = np.atleast_2d(np.sqrt(pars['R'])) * np.random.normal(size=(chunksize,p)) 
    y[idx] = x[idx].dot(pars['C'].T) + eps[idx] 
    if mmap:
        del y # releases RAM, forces flush to disk
        y = np.memmap(data_path+'y', dtype=np.float, mode='r+', shape=(T,p))      
        del eps # releases RAM, forces flush to disk
        eps = np.memmap(data_path+'eps', dtype=np.float, mode='r+', shape=(T,p))      
idx = (T//chunksize)*chunksize
eps[idx:] = np.atleast_2d(np.sqrt(pars['R'])) * np.random.normal(size=(T-(T//chunksize)*chunksize,p))
y[idx:] = x[(T//chunksize)*chunksize:].dot(pars['C'].T) + eps[idx:]
if mmap:
    del y # releases RAM, forces flush to disk
    y = np.memmap(data_path+'y', dtype=np.float, mode='r', shape=(T,p))   
    del eps # releases RAM, forces flush to disk
    eps = np.memmap(data_path+'eps', dtype=np.float, mode='r', shape=(T,p))      
    
Qs = []
for m in range(k+l):
    Qs.append(None)
    print('computing time-lagged covariance for lag ', str(m))
    if mmap:
        Q = np.memmap(data_path+'Qs_'+str(m), dtype=np.float, 
                      mode='w+', shape=(pa,pb))
    else:
        Q = np.empty((pa,pb))
    Q[:] = np.cov(y[m:m-(k+l),idx_a].T, y[:-(k+l),idx_b].T)[:pa,pa:]     
    if mmap:
        del Q
        Qs[m] = np.memmap(data_path+'Qs_'+str(m), dtype=np.float, 
                          mode='r', shape=(pa,pb))
    else:
        Qs[m] = Q
        
    if mmap:
        del y # releases RAM, forces flush to disk
        y = np.memmap(data_path+'y', dtype=np.float, mode='r', shape=(T,p))   
        del eps # releases RAM, forces flush to disk
        eps = np.memmap(data_path+'eps', dtype=np.float, mode='r', shape=(T,p))      
        
        
key_pars = {'C' : pars['C'],
            'R' : pars['R'],
            'k' : k, 'l' : l,
            'idx_a' : idx_a,
            'idx_b' : idx_b
           }
np.savez(data_path+'pars', 
         pars=key_pars)
np.savez(data_path+'x', x=x)


In [ ]:
idx_b = idx_a.copy()
plt.figure(figsize=(10*(k+l), 10))
for m in range(k+l):
    print('computing time-lagged covariance for lag ', str(m))
    
    plt.figure(figsize=(20,8))
    plt.subplot(1,3,1)
    #Qr[:] = (pars['C'][idx_a,:].dot(np.cov(x[m:m-(k+l),:].T, x[:-(k+l),:].T)[:n,n:]).dot(pars['C'][idx_b,:].T))
    #Qr = np.cov(pars['C'][idx_a,:].dot(x[m:m-(k+l),:].T) + eps[m:m-(k+l),idx_a].T, pars['C'][idx_b,:].dot(x[0:-(k+l),:].T) + eps[0:0-(k+l),idx_b].T)[:pa,pa:]
    Qr = np.cov(pars['C'][idx_a,:].dot(x[m:m-(k+l),:].T), pars['C'][idx_b,:].dot(x[0:-(k+l),:].T))[:pa,pa:]
    plt.imshow(Qr, interpolation='None')    

    plt.subplot(1,3,2)        
    plt.imshow(Qs[m], interpolation='None')    
    if mmap:
        Q = Qs[m]
        Qs[m] = None
        del Q
        Qs[m] = np.memmap(data_path+'Qs_'+str(m), dtype=np.float, 
                          mode='r', shape=(pa,pb))    

    plt.subplot(1,3,3)    
    plt.plot(Qs[m], Qr[:], 'b.')
    plt.title(str(np.corrcoef(Qs[m].reshape(-1), Qr.reshape(-1))[0,1]))
    if m == 0:
        plt.xlabel('not adding R to reconstruction!')
    plt.show()

# Loading data from disk (other default storage folder!)

In [ ]:

%matplotlib inline
if True:                                # make sure you want to execute this!
    import numpy as np
    import scipy as sp
    from sklearn.decomposition import PCA
    import matplotlib.pyplot as plt
    import glob, os, psutil, time

    data_path = '../fits/nonlinear_fitzhughnagumo/'

    pars=np.load(data_path+'pars.npz')['pars'].tolist()
    pars_est=np.load(data_path+'pars_est_full.npz')['arr_0'].tolist()['pars_est_full']
    pars_pca=np.load(data_path+'pars_pca.npz')['pars'].tolist()
    idx_a = pars['idx_a']
    idx_b = pars['idx_b']
    pa, pb = idx_a.size, idx_b.size
    x=np.load(data_path+'x.npz')['x']

    p,n = pars['C'].shape
    T = x.shape[0]
    k,l = pars['k'],pars['l']

    mmap, chunksize = True, np.min((T,p))
    Qs = []
    for m in range(k+l):
            Qs.append(np.memmap(data_path+'Qs_'+str(m), dtype=np.float, 
                          mode='r', shape=(pa,pb)))

    y = np.memmap(data_path+'y', dtype=np.float, mode='r', shape=(T,p))

In [ ]:
if False:                                # make sure you want to execute this!

    from scipy.io import savemat
    savemat(data_path+'parameters', {'pars' : np.load(data_path+'pars.npz')['pars'].tolist(),
                                     'pars_est_full':np.load(data_path+'pars_est_full.npz')['pars'].tolist(),
                                     'pars_est_partial':np.load(data_path+'pars_est_partial.npz')['pars'].tolist(),
                                     'pars_pca':pars_pca})


# check data

In [ ]:
if True:
    plt.figure(figsize=(20,10))
    plt.subplot(3,1,1)
    plt.plot(y[:, idx_a])
    plt.xlabel('t')
    plt.ylabel('y')
    plt.subplot(3,1,2)
    plt.plot(x)
    plt.xlabel('t')
    plt.ylabel('x')
    plt.subplot(3,1,3)
    plt.plot(T//2+np.arange(100,dtype=int), x[T//2+np.arange(100,dtype=int)])
    plt.xlabel('t')
    plt.ylabel('x')
    plt.show()
if mmap:
    del y # releases RAM, forces flush to disk
    y = np.memmap(data_path+'y', dtype=np.float, mode='r', shape=(T,p))



plt.figure(figsize=(20,10))
#if not mmap:
if True:
    pca = PCA()
    pca.fit(y)
    explained_variance_ratio_ = np.hstack((pca.explained_variance_ratio_, np.zeros(np.max((p-T,0)))))
    plt.plot(range(1,p+1), np.cumsum(explained_variance_ratio_)/np.sum(explained_variance_ratio_))
    plt.hold(True)
    plt.plot(np.linspace(0,p+1,np.min((20,p))), 
             np.cumsum(pca.explained_variance_ratio_[:np.min((20,p))])/np.sum(pca.explained_variance_ratio_), 
             'r--')
    plt.legend(('cum. var. expl.', 'first 20, x-axis rescaled'))

    pars_pca = {}
    pars_pca['C'] = pca.components_[:n].T
    pars_pca['Pi'] = np.diag(pca.explained_variance_[:n])    
    #np.savez(data_path+'pars_pca', 
    #     pars=pars_pca)
    
plt.xlabel('#eigvalue')
plt.title('% explained variance of PCA (if p not too large)')
plt.show()
print('noise variance / total variance: \n ', np.mean(pars['R']/np.var(y,axis=0)))

if p < 1000:
    L0 = np.cov(y.T)
    L0[np.diag_indices(p)] *= 0
    Lr = (pars['C']).dot(np.cov(x.T)).dot((pars['C']).T) + np.diag(pars['R'])
    Lr[np.diag_indices(p)] *= 0

    plt.figure(figsize=(15,8))
    plt.subplot(1,3,1)
    plt.imshow(L0, interpolation='None')
    
    plt.title('emp. cov matrix')
    plt.subplot(1,3,2)
    plt.imshow(Lr, interpolation='None')
    plt.title('est. cov matrix (C*cov(x)*C.T + cov(eps))')
    plt.subplot(1,3,3)
    plt.plot(L0[:], Lr[:], 'b.')
    plt.xlabel('emp.')
    plt.xlabel('est.')
    plt.show()

    print(np.var(y), pars['R'], 
          np.mean(np.abs(x.dot(pars['C'].T))), np.var(x.dot(pars['C'].T)))
#print(np.var(y, axis=0), np.var(pars['sqR']*eps, axis=0), np.mean(np.abs(x.dot(pars['C'].T)), axis=0))

#nonlinearity of latents
xp = x[0:T-1]
xf = x[1:T]
A_ls = np.linalg.lstsq(xp, xf)[0].T
xf_l = xp.dot(A_ls)
plt.figure(figsize=(20,np.ceil(n/2)*10))
for i in range(n):
    plt.subplot(np.ceil(n/2),2,i+1)
    plt.plot(xf_l[:,i], xf[:,i], 'k.')
    plt.hold(True)
    plt.plot(xf_l[:,i], xf_l[:,i], 'r.')
    plt.xlabel('x_pred_lin')
    plt.ylabel('x_emp')
    plt.axis('equal')
    mlim = np.max((np.max(np.abs(xf_l[:,i])),np.max(np.abs(xf[:,i]))))
    #plt.axis(1.1*np.array([-mlim,mlim,-mlim,mlim]))
    plt.title('non-linearity of x_'+str(i+1))
plt.show()
        
print('\n')
print(psutil.virtual_memory())
print(psutil.swap_memory())    


In [ ]:
plt.figure(figsize=(20, 20))
for m in range(k+l):
    plt.subplot(k+l,1,m)
    Q = np.empty((n,n))
    Q[:] = np.corrcoef(x[m:m-(k+l),:].T, x[:-(k+l),:].T)[:n,n:]
    plt.imshow(Q, interpolation='None')    
    plt.colorbar()
    plt.title('m' + str(m))
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import glob, os, psutil, time

os.chdir('../core')
import ssm_scripts, ssm_fit
from utility import get_subpop_stats, gen_data
from SSID_Hankel_loss import run_bad, plot_slim
os.chdir('../dev')

#np.random.seed(0)
#y -= np.mean(y,axis=0)
verbose=True

# settings for fitting algorithm
# settings for fitting algorithm
batch_size, max_zip_size, max_iter = 10, 10, 10
a, b1, b2, e = 0.0001, 0.9, 0.99, 1e-8
a_R = a

# create subpopulations
sub_pops = (np.arange(0,p), np.arange(0,p))
obs_pops = np.array([0,1])
obs_time = np.array([T//2, T])

obs_idx, idx_grp, co_obs, _, _, _, Om, _, _ = \
    get_subpop_stats(sub_pops=sub_pops, p=p, verbose=False)

pars_init='default'        
t = time.time()
pars_init, pars_est, traces = run_bad(k=k,l=l,n=n,y=y, Qs=Qs,Om=Om,idx_a=idx_a, idx_b=idx_b,
                                      sub_pops=sub_pops,idx_grp=idx_grp,co_obs=co_obs,obs_idx=obs_idx,
                                      obs_pops=obs_pops,obs_time=obs_time,
                                      linearity=linearity,stable=stable,init=pars_init,
                                      alpha=a,alpha_R=a_R,b1=b1,b2=b2,e=e,max_iter=max_iter,batch_size=batch_size,
                                      verbose=verbose, sym_psd=sym_psd, max_zip_size=max_zip_size)

print('fitting time was ', time.time() - t, 's')
print('\n')
print(psutil.virtual_memory())
print(psutil.swap_memory())

plot_slim(Qs,k,l,pars_est,idx_a,idx_b,traces,mmap,data_path)
  

In [ ]:
# settings for fitting algorithm
batch_size, max_zip_size, max_iter = 10, 100, 100
a, b1, b2, e = 0.001, 0.9, 0.99, 1e-8
a_R = 0.001

t = time.time()
_, pars_est, traces = run_bad(k=k,l=l,n=n,y=y, Qs=Qs,Om=Om,idx_a=idx_a, idx_b=idx_b,
                                      sub_pops=sub_pops,idx_grp=idx_grp,co_obs=co_obs,obs_idx=obs_idx,
                                      obs_pops=obs_pops,obs_time=obs_time,
                                      linearity=linearity,stable=stable,init=pars_est,
                                      alpha=a,alpha_R=a_R,b1=b1,b2=b2,e=e,max_iter=max_iter,batch_size=batch_size,
                                      verbose=verbose, sym_psd=sym_psd, max_zip_size=max_zip_size)

print('fitting time was ', time.time() - t, 's')
print('\n')

plot_slim(Qs,k,l,pars_est,idx_a,idx_b,traces,mmap,data_path)

In [ ]:
# settings for fitting algorithm
batch_size, max_zip_size, max_iter = 100, 3, 10
a, b1, b2, e = 0.001, 0.9, 0.99, 1e-8
a_R = 0.001

t = time.time()
_, pars_est, traces = run_bad(k=k,l=l,n=n,y=y, Qs=Qs,Om=Om,idx_a=idx_a, idx_b=idx_b,
                                      sub_pops=sub_pops,idx_grp=idx_grp,co_obs=co_obs,obs_idx=obs_idx,
                                      obs_pops=obs_pops,obs_time=obs_time,
                                      linearity=linearity,stable=stable,init=pars_est,
                                      alpha=a,alpha_R=a_R,b1=b1,b2=b2,e=e,max_iter=max_iter,batch_size=batch_size,
                                      verbose=verbose, sym_psd=sym_psd, max_zip_size=max_zip_size)

print('fitting time was ', time.time() - t, 's')
print('\n')

plot_slim(Qs,k,l,pars_est,idx_a,idx_b,traces,mmap,data_path)

In [ ]:
# settings for fitting algorithm
batch_size, max_zip_size, max_iter = 100, 10, 100
a, b1, b2, e = 0.0001, 0.9, 0.99, 1e-8
a_R = 0.0001

t = time.time()
_, pars_est, traces = run_bad(k=k,l=l,n=n,y=y, Qs=Qs,Om=Om,idx_a=idx_a, idx_b=idx_b,
                                      sub_pops=sub_pops,idx_grp=idx_grp,co_obs=co_obs,obs_idx=obs_idx,
                                      obs_pops=obs_pops,obs_time=obs_time,
                                      linearity=linearity,stable=stable,init=pars_est,
                                      alpha=a,alpha_R=a_R,b1=b1,b2=b2,e=e,max_iter=max_iter,batch_size=batch_size,
                                      verbose=verbose, sym_psd=sym_psd, max_zip_size=max_zip_size)

print('fitting time was ', time.time() - t, 's')
print('\n')

plot_slim(Qs,k,l,pars_est,idx_a,idx_b,traces,mmap,data_path)

In [ ]:
plt.figure(figsize=(10, 10))
for m in range(1):
    print('computing time-lagged covariance for lag ', str(m))
    
    Qf = np.empty((pa,pb))
    Qf[:] = np.cov(y[m:m-(k+l),idx_a].T, y[:-(k+l),idx_b].T)[:pa,pa:]    
    
    Q = np.empty((pa,pb))
    Q[:] = (pars_est['C'][idx_a,:].dot(pars_est['X'][:n,:]).dot(pars_est['C'][idx_b,:].T))
    plt.plot(Qf[:], Q[:], 'r.')
    plt.hold(True)
    plt.title('corr. est. us:' + str(np.corrcoef(Qf.reshape(-1), Q.reshape(-1))[0,1]))
    
    Q = np.empty((pa,pb))
    Q[:] = (pars_pca['C'][idx_a,:].dot(pars_pca['Pi']).dot(pars_pca['C'][idx_b,:].T))
    plt.plot(Qf[:], Q[:], 'b.')
    
    plt.xlabel('corr. est. PCA:' + str(np.corrcoef(Qf.reshape(-1), Q.reshape(-1))[0,1]))
    plt.show()

In [ ]:
from scipy.io import savemat
savemat('../fits/'+'parameters', {'pars' : pars,
                                 'pars_est_full':pars_est,
                                 'pars_est_partial':{}, 
                                 'pars_pca':pars_pca})


In [ ]:
X = np.zeros(((k+l)*n,n))
for m in range(k+l):
    X[m*n:(m+1)*n,:] = np.cov(x[m:m-(k+l),:].T, x[:-(k+l),:].T)[:n,n:]

pars_true= { 'C': pars['C'],
             'X': X,
             'R': pars['R']}
# settings for fitting algorithm
batch_size, max_zip_size, max_iter = 1, 100, 1000
a, b1, b2, e = 0.0001, 0.9, 0.99, 1e-8
a_R = 0.0001
linearity, stable, sym_psd = 'False', False, False

# create subpopulations
sub_pops = (np.arange(0,p), np.arange(0,p))

obs_idx, idx_grp, co_obs, _, _, _, Om, _, _ = \
    get_subpop_stats(sub_pops=sub_pops, p=p, verbose=False)

pars_init='default'        
t = time.time()
_, pars_est, traces = run_bad(k=k,l=l,n=n,y=y, Qs=Qs,Om=Om,idx_a=idx_a, idx_b=idx_b,
                                      sub_pops=sub_pops,idx_grp=idx_grp,co_obs=co_obs,obs_idx=obs_idx,
                                      linearity=linearity,stable=stable,init=pars_true,
                                      alpha=a,alpha_R=a_R,b1=b1,b2=b2,e=e,max_iter=max_iter,batch_size=batch_size,
                                      verbose=verbose, sym_psd=sym_psd, max_zip_size=max_zip_size)

print('fitting time was ', time.time() - t, 's')
print('\n')
print(psutil.virtual_memory())
print(psutil.swap_memory())

plot_slim(Qs,k,l,pars_est,idx_a,idx_b,traces,mmap,data_path)


# Wong and Wang's dynamics

\begin{align}
\frac{d s_i}{dt} = - \frac{s_i}{\tau_s} + (1 - s_i) \gamma H_i \\
H_i = \frac{a x_i -b}{1 - \exp(-d(a x_i -b))} \\
x_1 = J_{N,11}s_1 - J_{N,12} s_2 + I_0 + I_1 \\
x_2 = J_{N,22}s_2 - J_{N,21} s_1 + I_0 + I_2 \\
I_i = J_{A,ext} \mu_0 (1 \pm c)
\end{align}
Parameters: 
- $i \in \{1,2 \}$, 
- $a = 270 (VnC)^{-1}$, 
- $b = 108 Hz$,
- $d = 0.154s$, 
- $\gamma = 0.641$, 
- $\tau_s = 100ms$, 
- $J_{N,11} = J_{N,22} =  0.2608 nA$,
- $J_{N,12} = J_{N,21} =  0.0497 nA$,
- $J_{A,ext} = 0.00052 nA \cdot{} Hz^{-1}$,
- $\mu_0 = 30 Hz$

Remarks: marginally useful, as beset with strong attractors, preventing arbitrary simulation lengths


In [ ]:
%matplotlib inline

import numpy as np
import scipy as sp
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import glob, os, psutil, time

p,n,m,T = 1000,2,2,1000
k,l =  3,3
pars = {}

# I/O matter
mmap, chunksize = True, np.min((2*T,10000))
#data_path, save_file = '/media/marcel/636f7b46-1fd1-4600-b69e-86d2ed82002c/stitching/hankel/', 'test'
data_path, save_file = '../fits/', 'test'
pa, pb = np.min((p, 1000)), np.min((p, 1000))
idx_a, idx_b = np.sort(np.random.choice(p,pa,replace=False)), np.sort(np.random.choice(p,pb,replace=False))
verbose=True

# experimental parameters
I0 = 0
c = 0

# model parameters
pars = {}
pars['a'] = 270. * 10e9 # (VnC)^{-1} 
pars['b'] = 108. # Hz
pars['d'] = 0.154 # s 
pars['gamma'] = 0.641
pars['tau'] = 0.1 # s
pars['J'] = np.array([[0.2608,-0.0497],[-0.0497,0.2608]]) * 10e-9 # nA
pars['J_ext'] = 0.00052  * 10e-9 # nA Hz^{-1}
pars['mu'] = 30. # Hz
pars['I'] = pars['J_ext'] * pars['mu'] * (1 + np.array([1,-1]) * c)

# functions
def H(x):
    return (pars['a']*x-pars['b']) / (1-np.exp(-pars['d']*(pars['a']*x-pars['b'])))
def x(s):
    return pars['J'].dot(s) + pars['I'] + I0
    
# diff. eq.
def dsdt(s,t):
    return -s/pars['tau'] + (1-s)*pars['gamma']*H(x(s))

ts = np.linspace(0,10, 1000)
s0 = np.zeros(2)
N = 30
S1, S2 = np.meshgrid(np.linspace(0,1,N), np.linspace(0,1,N))
dS1, dS2 = np.zeros((N,N)), np.zeros((N,N))

for i in range(N):
    for j in range(N):
        tmp = dsdt(np.array((S1[i,j], S2[i,j])),0)
        dS1[i,j], dS2[i,j] = tmp[0], tmp[1]

plt.figure(figsize=(20,20))
plt.quiver(S1, S2, dS1, dS2, 
           edgecolor='k', linewidth=.5)

plt.show()